<a id="title_ID"></a>
# JWST Pipeline Validation Testing Notebook: spec2, assign_wcs step

<span style="color:red"> **Instruments Affected**</span>: NIRSpec 

Tested on CV3 data

### Table of Contents
<div style="text-align: left"> 

<br> [Imports](#imports_ID) <br> [Introduction](#intro_ID) <br> [Testing Data Set and Variable Setup](#data_ID) <br> [Run PTT, i.e. run JWST pipeline and assign_wcs validation tests](#pipeline_ID) <br> [About This Notebook](#about_ID) <br>

</div>

<a id="imports_ID"></a>
# Imports
The library imports relevant to this notebook are aready taken care of by importing PTT.

* astropy.io for opening fits files
* jwst.module.PipelineStep is the pipeline step being tested
* matplotlib.pyplot.plt to generate plot

NOTE: This notebook assumes that the pipeline version to be tested is already installed and its environment is activated.

To be able to run this notebook you need to install nptt. This will be installed with pip. If all goes well you will be able to import PTT.

[Top of Page](#title_ID)

In [2]:
import os
import matplotlib
from matplotlib import pyplot as plt

# make sure to always have the latest version of PTT
import subprocess
subprocess.run(['pip', 'install', 'git+https://github.com/spacetelescope/nirspec_pipe_testing_tool@master'])
import nirspec_pipe_testing_tool as nptt

<a id="intro_ID"></a>
# Introduction

We compared Institute's pipeline product of the assign_wcs step with the intermediary products from the ESA pipeline, which is completely independent from the Institute's. We calculated the relative difference and expected it to be equal to or less than computer precision:  relative_difference = absolute_value( (ESA - ST)/ESA )  <= 1x10^-7. 

For the test to be considered PASSED, every single slit (for FS data), slitlet (for MOS data) or slice (for IFU data) in the input file has to pass. If there is any failure, the whole test will be considered as FAILED. 

The input file is defined in the variable ```input_file``` (see section <br> [Testing Data Set and Variable Setup](#data_ID)).

This test has consistently passed since build 7.3, hence, we are now using our previous intermediary files as benchmark instead of ESA's data.  (STILL NOT SURE ABOUT THIS)

Step description: https://jwst-pipeline.readthedocs.io/en/latest/jwst/assign_wcs/main.html

Pipeline code: https://github.com/spacetelescope/jwst/tree/master/jwst/assign_wcs


### Calibration WG Requested Algorithm: 

A short description and link to the page: https://outerspace.stsci.edu/display/JWSTCC/Vanilla+Spectral+GWCS+Information 


### Defining Term
Acronymns used un this notebook:

pipeline: calibration pipeline

spec2: spectroscopic calibration pipeline level 2b

PTT: NIRSpec pipeline testing tool (https://github.com/spacetelescope/nirspec_pipe_testing_tool)


[Top of Page](#title_ID)

<a id="data_ID"></a>
# Testing Data Set and Variable Setup

All testing data is from the CV3 campaign. We chose these files because this is our most complete data set, i.e. all modes and filter-grating combinations.

Data used was for testing:
FS - FULLFRAME, G140M, OPAQUE
FS - ALLSLITS, G235H, F170LP 
IFU - G395H, OPAQUE  and  PRISM, CLEAR
MOS - G140M, F100LP 

The FS testing data can be located at:
```/grp/jwst/wit4/nirspec_vault/pipe_testing_tool/PTT_FS_benchmark_run```

In that directory you will find:
- NRSV84600010001P0000000002101_4_491_SE_2016-01-17T17h34m08.fits (which is what we call the raw_data_file)
- gain_scale_NRS1.fits (which is the final output of cal_detector1 and the input file for your test)
- results_491 (directory that contains results of my tests, in case you want to compare your results)
- PTT_config.cfg  (a sample of a configuration file for PTT, in case you want to compare with the one you create in the test)

If you want to DownlThe only file you need to Download is gain_scale_NRS1.fits, since this is the input file for the test.



Set the following variables to the file you are interested in testing:

- output_directory = path where you want intermediary files and plots to be saved in

- input_file = this is the count rate file, e.g. jwtest1_NRS1_rate.fits

- mode_used = supported modes are: FS, MOS, IFU, BOTS, MOS_sim, dark, image, confirm, taconfirm, wata, msata, focus, mimf

- raw_data_root_file = CV3 raw data file

- run_calwebb_spec2 = step to run, possible values are True (for all steps) or name of the step: "assign_wcs", "bkg_subtract", "imprint_subtract", "msa_flagging", "extract_2d", "flat_field", "srctype", "pathloss", "barshadow", "photom", "resample_spec", "cube_build", "extract_1d"


The following variables only need to be set if needing to change the default:

- data_directory = path where PTT will look for the input file, only needed if different from output_directory

- local_pipe_cfg_path = path to local configuration files  

- msa_conf_name = /path_to_corresponding_MSA_shutter_configuration_file/V9621500100101_short_msa.fits

- wcs_threshold_diff = permitted difference between comparison file and pipeline output

- save_plots = If True, diagnostig plots will be saved in the output directory 

- dflat_path = /grp/jwst/wit4/nirspec/CDP3/04_Flat_field/4.2_D_Flat/nirspec_dflat

- sflat_path = /grp/jwst/wit4/nirspec/CDP3/04_Flat_field/4.3_S_Flat/FS/nirspec_FS_sflat

- fflat_path = /grp/jwst/wit4/nirspec/CDP3/04_Flat_field/4.1_F_Flat/FS/nirspec_FS_fflat

NOTE: For sflat and fflats variables, the mode should be changed accordingly, i.e. nirspec_FS_sflat, nirspec_IFU_sflat, nirspec_MOS_sflat


[Top of Page](#title_ID)

In [3]:
output_directory = '/Users/pena/Documents/PyCharmProjects/nirspec/pipeline/testing_data/FS_FULL_FRAME/G140M_opaque/'
input_file = 'gain_scale_NRS1.fits'
mode_used = 'FS'
run_calwebb_spec2 = 'assign_wcs'
raw_data_root_file = 'NRSV84600010001P0000000002101_4_491_SE_2016-01-17T17h34m08.fits'
msa_conf_name = '/path_to_corresponding_MSA_shutter_configuration_file/V9621500100101_short_msa.fits'

# These are examples of additional variables you may want to change
#data_directory = '/Users/pena/Documents/PyCharmProjects/nirspec/pipeline/testing_data/FS_FULL_FRAME/G140M_opaque/'
#comparison_file_path = 'my_benchmark_file.fits'
#local_pipe_cfg_path = 'pipe_source_tree_code'
#dflat_path = '/grp/jwst/wit4/nirspec/CDP3/04_Flat_field/4.2_D_Flat/nirspec_dflat_nrs1_f_01.03.fits
#sflat_path = '/grp/jwst/wit4/nirspec/CDP3/04_Flat_field/4.3_S_Flat/FS/nirspec_FS_sflat_G140M_OPAQUE_FLAT1_nrs1_f_01.01.fits'
#fflat_path = '/grp/jwst/wit4/nirspec/CDP3/04_Flat_field/4.1_F_Flat/FS/nirspec_FS_fflat_F100LP_01.01.fits'
#wcs_threshold_diff = 1.0e-7
#save_wcs_plots = True


<a id="pipeline_ID"></a>
# Run PTT, i.e. run JWST pipeline and assign_wcs validation tests

The pipeline can be run from the command line in two variants: full or per step.

Tu run the spec2 pipeline in full use the command: 

$ strun jwst.pipeline.Spec2Pipeline jwtest_rate.fits

Tu only run the assign_wcs step, use the command:

$ strun jwst.assign_wcs.AssignWcsStep jwtest_rate.fits

NIRSpec TA data will be run through the cal_detector1 and the imaging2 pipelines. The imaging pipeline can be run with the following fommand:

$ strun jwst.pipeline.Image2Pipeline jwtest_rate.fits

These options are also callable from a script with the testing environment active. The Python call for running the pipeline in full or by step are:

$\gt$ from jwst.pipeline.calwebb_spec2 import Spec2Pipeline

$\gt$ Spec2Pipeline.call(jwtest_rate.fits)
 
or
 
$\gt$ from jwst.assign_wcs.assign_wcs_step import AssignWcsStep
 
$\gt$ AssignWcsStep.call(jwtest_rate.fits)

For the imaging pipeline the call would be as follows:

$\gt$ from jwst.pipeline.calwebb_image2 import Image2Pipeline

$\gt$ Image2Pipeline.call(jwtest_rate.fits)

PTT can run the spec2 pipeline either in full or per step, as well as the imaging pipeline in full. In this notebook we will use PTT to run the pipeline and the validation tests.

To run PTT you first need to create and edit the file that controls the tool. This configuration file tells PTT which data to test, the mode, and where to put results and plots, among other variables that can be modified from the default values (e.g. paths for the flat field reference files, acceptance threshold values for differences between pipeline product and benchmark files, modify a benchmark file, etc).

[Top of Page](#title_ID)

In [4]:
# with the variables previously defined, generate the PTT configuration file in the current directory
nptt.utils.mk_pttconfig_file.mk_ptt_cfg(output_directory, input_file, mode_used, raw_data_root_file,
                                        msa_conf_name=msa_conf_name, run_calwebb_spec2=run_calwebb_spec2)

# if all went well, you should have a PTT_config.cfg file in your output directory

 * PTT will save all test output plots in the output directory.

 * Script  mk_pttconfig_file.py  finished * 



After you created the PTT_config.cfg file, and you made sure all the variables in it are set as expected, run the pipeline with the following command

In [4]:
nptt.utils.run_PTT.run_PTT('report_assign_wcs', os.path.join(output_directory, 'PTT_config.cfg'))

Running PTT. This may take a while...
-> The detector used added to the html report name:  report_assign_wcs_NRS1.html
Moving PTT html report to working directory
Done.

 * Script  run_PTT.py  finished * 



After PTT is done, an html report will have been created in the output directory. This report contains the passed, skipped, and failed tests. All diagnostoc plots and output files (such as the intermediary step output) will also be saved in the output directory. There will also be a pipeline log (which contains the screen output from running the pipeline) and a PTT log (which contains the screen output from PTT), as well as a text file with the time it took the pipeline to run the step(s). 


<a id="about_ID"></a>
## About this Notebook
**Author:** Maria A. Pena-Guerrero, Staff Scientist II - Systems Science Support, NIRSpec
<br>**Updated On:** 04/09/2020

[Top of Page](#title_ID)
<img style="float: right;" src="./stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="stsci_pri_combo_mark_horizonal_white_bkgd" width="200px"/> 